Convert the JSONL to huggingface format
https://docs.unsloth.ai/models/tutorials-how-to-fine-tune-and-run-llms/tutorial-how-to-finetune-llama-3-and-use-in-ollama

In [20]:
import unsloth
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype =  None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.9.7: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    NVIDIA GeForce RTX 5060 Ti. Num GPUs = 1. Max memory: 15.928 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.8.0+cu129. CUDA: 12.0. CUDA Toolkit: 12.9. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Customize fine-tuning, experiment to get better results

In [21]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Transfer the JSONL file into a Huggingface dataset

In [22]:
import json
from datasets import Dataset

# Load training data from JSONL file
training_data = []
with open("Data/socratic_questions.jsonl", "r") as f:
    for line in f:
        training_data.append(json.loads(line))

# Convert to Hugging Face dataset format
def format_instruction(example):
    return {
        "text": f"### Short explanation:\n{example['short_explanation']}\n\n### Response:\n{example['question']}"
    }

dataset = Dataset.from_list(training_data)
dataset = dataset.map(format_instruction)
print(dataset.column_names)

Map:   0%|          | 0/4995 [00:00<?, ? examples/s]

['claim', 'short_explanation', 'verdict', 'question', 'text']


Merge the claim, short explanation and verdict into one prompt. The output is the question.

In [23]:
from unsloth import to_sharegpt

dataset = to_sharegpt(
    dataset,
    merged_prompt=\
        "[[The intial claim was: {claim}]]"\
        "[[The final verdict was: {verdict}]]"\
        "[[The justification for this verdict was: {short_explanation}]]",
    conversation_extension=3,  # Select more to handle longer conversations    
    output_column_name="question",
)

Merging columns:   0%|          | 0/4995 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/4995 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/4995 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/4995 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/4995 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/4995 [00:00<?, ? examples/s]

In [24]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

Unsloth: Standardizing formats (num_proc=12):   0%|          | 0/4995 [00:00<?, ? examples/s]

In [28]:
chat_template = """Below is the initial claim, the final fact checked verdict, and the justification for this verdict.
The output is a Socratic question that challenges the claim and encourages critical thinking.
>>> Claim, jusitification and verdict:
{INPUT}
>>> Socratic question?
{OUTPUT}"""

from unsloth import apply_chat_template

# from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
#     # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)



Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/4995 [00:00<?, ? examples/s]

Train the model

In [31]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/4995 [00:00<?, ? examples/s]

RuntimeError: One of the subprocesses has abruptly died during map operation.To debug the error, disable multiprocessing.